In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim

from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [5]:
# Load your dataset
df = pd.read_csv(r"D:\project grad linex\pair_pas_matrix (1).csv")


# Prepare features and labels
X = df.drop(columns=["label"])
y = df["label"]


In [6]:
#df = pd.read_csv('../dataset/clean.csv')
df.head()

,A_HALLMARK_APOPTOSIS,A_HALLMARK_DNA_REPAIR,A_HALLMARK_G2M_CHECKPOINT,A_HALLMARK_HYPOXIA,A_HALLMARK_MTOR_SIGNALING,A_HALLMARK_TNFA_SIGNALING_VIA_NFKB,A_HALLMARK_P53_PATHWAY,A_HALLMARK_E2F_TARGETS,A_HALLMARK_MYOGENESIS,A_HALLMARK_OXIDATIVE_PHOSPHORYLATION,...,B_HALLMARK_PANCREAS_BETA_CELLS,B_HALLMARK_PEROXISOME,B_HALLMARK_PI3K_AKT_MTOR_SIGNALING,B_HALLMARK_PROTEIN_SECRETION,B_HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY,B_HALLMARK_SPERMATOGENESIS,B_HALLMARK_UV_RESPONSE_DN,B_HALLMARK_UV_RESPONSE_UP,B_HALLMARK_XENOBIOTIC_METABOLISM,label
0,0.224941,-0.007462,-0.039723,-0.043391,0.006737,-0.092991,-0.091991,0.087857,0.190934,0.032228,...,-0.111067,-0.182822,-0.139949,-0.053313,-0.227537,-0.173107,-0.100992,-0.157266,-0.173373,1
1,0.224941,-0.007462,-0.039723,-0.043391,0.006737,-0.092991,-0.091991,0.087857,0.190934,0.032228,...,-0.079203,-0.360736,-0.060379,-0.026245,-0.301333,-0.143247,0.100665,-0.179199,-0.029461,1
2,0.224941,-0.007462,-0.039723,-0.043391,0.006737,-0.092991,-0.091991,0.087857,0.190934,0.032228,...,-0.050723,-0.276856,-0.229361,-0.117909,-0.276944,-0.146585,-0.209756,-0.074772,-0.028250,1
3,0.224941,-0.007462,-0.039723,-0.043391,0.006737,-0.092991,-0.091991,0.087857,0.190934,0.032228,...,0.524971,0.580937,0.713780,0.312925,0.935355,0.785940,0.475780,0.613535,0.403671,1
4,0.224941,-0.007462,-0.039723,-0.043391,0.006737,-0.092991,-0.091991,0.087857,0.190934,0.032228,...,1.280787,1.520947,1.634686,0.443156,2.220325,1.413702,1.280451,1.436004,0.662154,1


In [84]:
X = df.drop('gdp', axis=1).values
y = df['gdp'].values

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

# STEP 1: First split → training/validation (85%) and holdout test (15%)
X_train_val, X_test_final, y_train_val, y_test_final = train_test_split(
    X, y, test_size=0.15, stratify=y, random_state=42
)

# STEP 2: Split training/validation → train (70%) and val (15%) of original
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, stratify=y_train_val, random_state=42
)
# (0.1765 × 85% ≈ 15% of original dataset)

# STEP 3: Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test_final = scaler.transform(X_test_final)

# STEP 4: Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test_final, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_final.values, dtype=torch.float32).unsqueeze(1)


In [25]:
# Create model instance
model = MLP(X_train_tensor.shape[1])

# Weight the loss to focus more on class 0
# Try different values like 1.5, 2.0, 3.0 to improve
class_weights = torch.tensor([2.0])
criterion = nn.BCELoss(weight=class_weights)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Create model
model = MLP(X_train_tensor.shape[1])  # dynamically uses correct input size
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [28]:
# Training the model with validation
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validate
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")


Epoch [1/50] - Train Loss: 1.1183 | Val Loss: 1.1548
Epoch [2/50] - Train Loss: 1.1017 | Val Loss: 1.1551
Epoch [3/50] - Train Loss: 1.1098 | Val Loss: 1.1552
Epoch [4/50] - Train Loss: 1.1135 | Val Loss: 1.1553
Epoch [5/50] - Train Loss: 1.1013 | Val Loss: 1.1546
Epoch [6/50] - Train Loss: 1.1001 | Val Loss: 1.1539
Epoch [7/50] - Train Loss: 1.0907 | Val Loss: 1.1529
Epoch [8/50] - Train Loss: 1.0873 | Val Loss: 1.1525
Epoch [9/50] - Train Loss: 1.0846 | Val Loss: 1.1528
Epoch [10/50] - Train Loss: 1.0802 | Val Loss: 1.1534
Epoch [11/50] - Train Loss: 1.0769 | Val Loss: 1.1532
Epoch [12/50] - Train Loss: 1.0734 | Val Loss: 1.1527
Epoch [13/50] - Train Loss: 1.0783 | Val Loss: 1.1521
Epoch [14/50] - Train Loss: 1.0789 | Val Loss: 1.1513
Epoch [15/50] - Train Loss: 1.0631 | Val Loss: 1.1504
Epoch [16/50] - Train Loss: 1.0714 | Val Loss: 1.1509
Epoch [17/50] - Train Loss: 1.0597 | Val Loss: 1.1512
Epoch [18/50] - Train Loss: 1.0532 | Val Loss: 1.1520
Epoch [19/50] - Train Loss: 1.0505 | 

In [31]:
# ✅ Final Holdout Evaluation (on completely unseen data)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_labels = (test_outputs > 0.5).float()

# Evaluation metrics
print("✅ Final Holdout Evaluation Results:")
print("Accuracy:", accuracy_score(y_test_tensor, test_labels))
print("F1 Score:", f1_score(y_test_tensor, test_labels))
print("ROC-AUC:", roc_auc_score(y_test_tensor, test_outputs))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_tensor, test_labels))
print("\nClassification Report:\n", classification_report(y_test_tensor, test_labels))


✅ Final Holdout Evaluation Results:
Accuracy: 0.7132867132867133
F1 Score: 0.8110599078341013
ROC-AUC: 0.7662131519274376

Confusion Matrix:
 [[14 31]
 [10 88]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.58      0.31      0.41        45
         1.0       0.74      0.90      0.81        98

    accuracy                           0.71       143
   macro avg       0.66      0.60      0.61       143
weighted avg       0.69      0.71      0.68       143



In [29]:
# Final test evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_labels = (y_pred > 0.5).float()

# Evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

print("Test Accuracy:", accuracy_score(y_test_tensor, y_pred_labels))
print("Test F1 Score:", f1_score(y_test_tensor, y_pred_labels))
print("Test ROC-AUC:", roc_auc_score(y_test_tensor, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_tensor, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test_tensor, y_pred_labels))


Test Accuracy: 0.7412587412587412
Test F1 Score: 0.8340807174887892
Test ROC-AUC: 0.8149659863945579

Confusion Matrix:
 [[13 32]
 [ 5 93]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.72      0.29      0.41        45
         1.0       0.74      0.95      0.83        98

    accuracy                           0.74       143
   macro avg       0.73      0.62      0.62       143
weighted avg       0.74      0.74      0.70       143



In [ ]:
mlp.eval() 

In [ ]:
with torch.no_grad():
    outputs = mlp(test_data)
    predicted_labels = outputs.squeeze().tolist()

predicted_labels = np.array(predicted_labels)
test_targets = np.array(test_targets)

mse = mean_squared_error(test_targets, predicted_labels)
r2 = r2_score(test_targets, predicted_labels)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)